In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering

import pandas as pd
import numpy as np

from operator import add
import math

In [2]:
# Spark Constants
APP_NAME = 'assignment1'
MASTER = 'local[*]'

#random hyper planes
# Data Columns
COLUMN_USER_ID = 'userId'
COLUMN_MOVIE_ID = 'movieId'
COLUMN_RATING = 'rating'
COLUMN_TIMESTAMP = 'timestamp'

# Input Constants
MOVIES_INPUT_FILE = 'movies.csv'
TAGS_INPUT_FILE = 'tags.csv'
RATINGS_INPUT_FILE = 'ratings.csv'

In [3]:
conf = SparkConf().setAppName(APP_NAME).setMaster(MASTER)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.appName(APP_NAME).master(MASTER).getOrCreate()

23/05/04 23:42:28 WARN Utils: Your hostname, pedro-duarte resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp2s0)
23/05/04 23:42:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/04 23:42:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/04 23:42:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
def create_item_matrix(dataset):
    return dataset.pivot(index=COLUMN_MOVIE_ID, columns=COLUMN_USER_ID, values=COLUMN_RATING).fillna(0)

In [6]:
def cluster_ratings(dataset):
    X = dataset.values
    X = StandardScaler().fit(X).transform(X)

    return AgglomerativeClustering(n_clusters=None, distance_threshold=200).fit_predict(X)

In [7]:
def predict_rating(item_matrix, clusters, user_id, movie_id):
    if movie_id not in item_matrix.index.to_list(): return 0

    movie_idx = item_matrix.index.get_loc(movie_id)
    movie_to_predict = item_matrix.loc[movie_id]

    similar_movies = item_matrix[clusters == clusters[movie_idx]]

    if user_id not in similar_movies.columns.to_list(): return movie_to_predict.mean()
    similar_movies = similar_movies[similar_movies[user_id] != 0]

    distances = np.dot(movie_to_predict, similar_movies.T)/(np.linalg.norm(movie_to_predict)*np.linalg.norm(similar_movies.T))
    
    user_ratings = similar_movies.get(user_id)

    if user_ratings is None: return movie_to_predict.mean()

    ratings_product = np.dot(user_ratings, distances).sum()
    user_ratings_distance_total = distances.sum()

    if user_ratings_distance_total == 0: return movie_to_predict.mean()

    rating_prediction = ratings_product/user_ratings_distance_total

    return rating_prediction

In [8]:
ds = create_item_matrix(ratings)
clusters = cluster_ratings(ds)

predict_rating(ds, clusters, 416, 319)

2.020764511562058

# 2.1

In [9]:
ratings_count = ratings.shape[0]

ratings_test_count = math.ceil(ratings_count*.1)

# Shuffle
ratings = ratings.sample(frac = 1)

# Set 10% of ratings for testing
ratings_test = ratings[ratings_test_count:]
ratings_train = ratings[:ratings_test_count]


In [10]:
train_ds = create_item_matrix(ratings_train)
train_clusters = cluster_ratings(train_ds)

In [11]:
deviations = sc.parallelize(ratings_test.values) \
  .map(lambda v: predict_rating(train_ds, train_clusters, v[0], v[1])) \
  .reduce(add)

print("Deviation:", deviations/ratings_test_count)

23/05/04 23:43:03 WARN TaskSetManager: Stage 0 contains a task of very large size (1371 KiB). The maximum recommended task size is 1000 KiB.


Deviation: 22.58084157562842
